In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
geckodriver_path = r"C:\Users\alkan\OneDrive\Masaüstü\twitter\driver\geckodriver.exe"

In [227]:
driver = webdriver.Firefox()
driver.get('https://twitter.com/login')

In [155]:
UserNames = []
Contents = []
Replies = []
Reposts = []
Likes = []
Bookmarks = []
Views = []
Dates = []
IsBlue = []

In [218]:
MyList = [Tweet_Urls, UserNames, Contents, Replies, Reposts, Likes, Bookmarks, Views, Dates, IsBlue]

In [247]:
for sublist in MyList:
    sublist.clear()

In [248]:
print(MyList)

[[], [], [], [], [], [], [], [], [], []]


In [4]:
max_tweets = 10  # Alınacak maksimum tweet sayısı
current_tweet = 0  # İşlenen tweet sayısı

while current_tweet < max_tweets:
    # Anasayfadaki tüm tweetleri al
    articles = WebDriverWait(driver, 10).until(
        EC.visibility_of_all_elements_located((By.XPATH, "//article[@data-testid='tweet']")))

    # Her bir tweet için işlemleri yap
    for article in articles:
        tweet = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//div[@data-testid='tweetText']")))
        tweet.click()  # Her bir tweete tıkla
        
        # Tweet bilgilerini al
        usernames_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//div[@data-testid='User-Name']")))
        usernames_text = usernames_element.text
        username_split = usernames_text.split("\n")
        UserNames.append(username_split[1])  # Kullanıcı adını ekle
        
        # Mavi hesap kontrolü
        username_id = usernames_element.get_attribute("id")
        try:
            blue = WebDriverWait(driver, 0.8).until(
                EC.visibility_of_element_located((By.XPATH, f"//div[@id='{username_id}']//span[@class='css-1qaijid r-bcqeeo r-qvutc0 r-poiln3 r-1awozwy r-xoduu5']")))
            IsBlue.append(True)  # Hesap mavi ise True değeri ekle
        except:
            IsBlue.append(False)  # Hesap mavi değilse False değeri ekle
        
        tweet_url = driver.current_url  # Tweet URL'sini al
        Tweet_Urls.append(tweet_url)
        
        content = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//div[@data-testid='tweetText']"))).text.replace('/n','')
        Contents.append(content)  # İçeriği ekle
        
        group = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//div[@role='group']")))
        group_values = group.get_attribute("aria-label")

        metrics = group_values.split(", ")
        for metric in metrics:
            count, metric_type = metric.split(" ", 1)
            count = int(count) 
            if metric_type == 'replies':
                Replies.append(count)
            elif metric_type == 'reposts':
                Reposts.append(count)
            elif metric_type == 'likes':
                Likes.append(count)
            elif metric_type == 'bookmarks':
                Bookmarks.append(count)
            elif metric_type == 'views':
                Views.append(count) 
            
        if not Replies:
            Replies.append(0)
        if not Reposts:
            Reposts.append(0)
        if not Likes:
            Likes.append(0)
        if not Bookmarks:
            Bookmarks.append(0)
        if not Views:
            Views.append(0)
        
        time = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//time"))).text
        Dates.append(time)  # Tarih bilgisini ekle
        
        driver.back()  # Bir önceki sayfaya geri dön
        sleep(1)  # Bir sonraki tweete geçmeden önce bir süre bekleyin
        
        current_tweet += 1  # İşlenen tweet sayısını artırın
        
        if current_tweet >= max_tweets:
            break  # Maksimum tweet sayısına ulaşıldığında döngüyü sonlandır
    
    scroll_value = 500  # Kaydırılacak piksel miktarı
    driver.execute_script(f"window.scrollBy(0, {scroll_value});")
    sleep(1)  # Ekranın aşağı kaydırılmasından sonra bir süre bekleyin

WebDriverException: Message: Failed to write request to stream


## Group sayılarına erişme

In [150]:
group = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//div[@role = 'group']")))
group_values = group.get_attribute("aria-label")


metrics = group_values.split(", ")
for metric in metrics:
    count, metric_type = metric.split(" ", 1)
    count = int(count) 
    if metric_type == 'replies':
        Replies.append(count)
    elif metric_type == 'reposts':
        Reposts.append(count)
    elif metric_type == 'likes':
        Likes.append(count)
    elif metric_type == 'bookmarks':
        Bookmarks.append(count)
    elif metric_type == 'views':
        Views.append(count)

if not Replies:
    Replies.append(0)
if not Reposts:
    Reposts.append(0)
if not Likes:
    Likes.append(0)
if not Bookmarks:
    Bookmarks.append(0)
if not Views:
    Views.append(0)